## Import libraries 

In [31]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

Read Excel with Pandas library 

In [8]:
df = pd.read_excel("datasets/final/dataset_linkedin.xlsx")
df.head()

,subject,body,from,date,label
0,"Julien, votre candidature a été envoyée à XRAY...",Votre candidature a été envoyée à XRAYS TRADIN...,jobs-noreply@linkedin.com,16/11/2025 10:57:31,1
1,"Julien, votre candidature a été envoyée à Equancy",Votre candidature a été envoyée à Equancy\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:55:57,1
2,"Julien, votre candidature a été envoyée à Inno...",Votre candidature a été envoyée à Innova Solut...,jobs-noreply@linkedin.com,12/11/2025 13:55:23,1
3,"Julien, votre candidature a été envoyée à Aude...",Votre candidature a été envoyée à Audensiel\n\...,jobs-noreply@linkedin.com,12/11/2025 13:54:50,1
4,"Julien, votre candidature a été envoyée à Extia",Votre candidature a été envoyée à Extia\n\n\n\...,jobs-noreply@linkedin.com,12/11/2025 13:53:25,1


## Basics EDA (appropriate in this context)

### 1. Analyzing the data  

In [19]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  195 non-null    object
 1   body     195 non-null    object
 2   from     195 non-null    object
 3   date     195 non-null    object
 4   label    195 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.7+ KB


### 2. Checking missing values 

In [20]:
df.isnull().sum()

subject    0
body       0
from       0
date       0
label      0
dtype: int64

### 3. Checking for the duplicates values 

In [21]:
df.nunique()

subject    137
body       195
from         1
date       195
label        4
dtype: int64

The last result let us understand that even if there is several labels, there is only one mail that is sending messages. Let's verify which one is it.

In [26]:
print(*set(df["from"])) # Set = drop duplicates, * = unpack the set 

jobs-noreply@linkedin.com


<b>FIRST RULE : VERIFY THAT THE MAIL IS COMING FROM jobs-noreply@linkedin.com TO ANALYSE IT</b>

## I wanna see for each label, which words is the most common in the subject :  

### number element labelised 1 

In [49]:
len(list(df.loc[df["label"] ==1, "subject"]))

92

In [51]:
words = (
    df.loc[df["label"] ==1, "subject"]
    .str.lower()                  # normalise
    .str.split()                  # separate into words
    .explode()                    # 1 word = 1 line
)

words.value_counts().head(15)

julien,        92
candidature    92
a              92
été            92
envoyée        92
à              92
votre          92
-               9
group           6
solutions       4
de              3
&               3
générale        3
direction       3
la              3
Name: subject, dtype: int64

In [53]:
df.loc[df["label"] ==1, "subject"].head()

0    Julien, votre candidature a été envoyée à XRAY...
1    Julien, votre candidature a été envoyée à Equancy
2    Julien, votre candidature a été envoyée à Inno...
3    Julien, votre candidature a été envoyée à Aude...
4      Julien, votre candidature a été envoyée à Extia
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"Prénom, votre candidature a été envoyée à"</b> appear for each element labelised 1 (see the word document to remind the labels meaning)

<b> SECOND RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Name, votre candidature a été envoyée à" IT MEANS THAT THE COMPANY JUST RECEIVE THE APPLICATION </b>